## Application: Efficiency <a class="anchor" id="efficiency"></a>

**Learning outcome:** by the end of this section, you will be able to apply concepts that allow you to perform fast and efficient calculations on NumPy arrays.

### Views on Arrays

NumPy attempts to not make copies of arrays. Many NumPy operations will produce a reference to an existing array, known as a "view", instead of making a whole new array. For example, indexing and reshaping provide a view of the same memory wherever possible.

In [ ]:
import numpy as np

arr = np.arange(8)
arr_view = arr.reshape(2, 4)

# Print the "view" array from reshape.
print('Before\n{}'.format(arr_view))

# Update the first element of the original array.
arr[0] = 1000

# Print the "view" array from reshape again,
# noticing the first value has changed.
print('After\n{}'.format(arr_view))

What this means is that if one array (`arr`) is modified, the other (`arr_view`) will also be updated :  the same memory is being shared. This is a valuable tool which enables the system memory overhead to be managed, which is particularly useful when handling lots of large arrays. The lack of copying allows for very efficient vectorized operations.

Remember, this behaviour is automatic in most of NumPy, so it requires some consideration in your code, it can lead to some bugs that are hard to track down. For example, if you are changing some elements of an array that you are using elsewhere, you may want to explicitly copy that array before making changes. If in doubt, you can always copy the data to a different block of memory with the `copy()` method.

For example:

In [ ]:
arr = np.arange(8)
arr_view = arr.reshape(2, 4).copy()

# Print the "view" array from reshape.
print('Before\n{}'.format(arr_view))

# Update the first element of the original array.
arr[0] = 1000

# Print the "view" array from reshape again,
# noticing the first value has changed.
print('After\n{}'.format(arr_view))

### Loops and Vectorised Operations

We will now explore calculation performance and consider efficiency in terms of processing time.

Firstly let's look at a simple processing time tool that is provided in notebooks; ``%%timeit`` :

In [ ]:
%%timeit 
x = range(500)

Repeat that, specifying only 100 loops and fastest time of 5 runs

In [ ]:
%%timeit -n 100 -r 5
x = range(500)

This gives us an easy way to evaluate performance for implementations ...

In [ ]:
rands = np.random.random(1000000).reshape(100, 100, 100)

In [ ]:
%%timeit -n 10 -r 5
overPointEightLoop = 0
for i in range(100):
    for j in range(100):
        for k in range(100):
            if rands[i, j, k] > 0.8:
                overPointEightLoop +=1

In [ ]:
%%timeit -n 10 -r 5
overPointEightWhere = rands[rands > 0.8].size

#### Efficiency: Summary of key  points

 * NumPy enables vectorised calculations that are fast:
   * Python loops are much slower.
 * NumPy can only use the system memory available to hold arrays:
   * very large arrays can result in a `MemoryError`.
 * Many NumPy operations return views on exisiting array:
   * a view shares memory with the original array,
   * changing array contents in place will affect all related views,
   * sometimes it is useful to explicitly `copy` arrays.

### Exercise: trapezoidal integration

In this exercise, you are tasked with implementing the simple trapezoid rule
formula for numerical integration. If we want to compute the definite integral

$$
     \int_{a}^{b}f(x)dx
$$

we can partition the integration interval $[a,b]$ into smaller subintervals. We then
approximate the area under the curve for each subinterval by calculating the area of
the trapezoid created by linearly interpolating between the two function values
at each end of the subinterval:

![Illustration of the trapezoidal rule](images/trapezoidal_rule.png)

For a pre-computed $y$ array (where $y = f(x)$ at discrete samples) the trapezoidal rule equation is:

$$
   \int_{a}^{b}f(x)dx\approx\frac{1}{2}\sum_{i=1}^{n}\left(x_{i}-x_{i-1}\right)\left(y_{i}+y_{i-1}\right).
$$

In pure python, this can be written as:

    def trapz_slow(x, y):
        area = 0.
        for i in range(1, len(x)):
            area += (x[i] - x[i-1]) * (y[i] + y[i-1])
        return area / 2

#### Part 1

Create two arrays $x$ and $y$, where $x$ is a linearly spaced array in the interval $[0, 3]$ of length 11, and $y$ represents the function $f(x) = x^2$ sampled at $x$.

#### Part 2

Use indexing (not a for loop) to find the 10 values representing $y_{i}+y_{i-1}$ for $i$ between 1 and 11.

Hint: What indexing would be needed to get all but the last element of the 1d array **``y``**. Similarly what indexing would be needed to get all but the first element of a 1d array.

#### Part 3

Write a function `trapz(x, y)`, that applies the trapezoid formula to pre-computed values, where `x` and `y` are 1-d arrays. The function should not use a for loop.

#### Part 4

Verify that your function is correct by using the arrays created in #1 as input to ``trapz``. Your answer should be a close approximation of $\int_0^3 x^2$ which is $9$.

#### Part 5 (extension)

``numpy`` and ``scipy.integrate`` provide many common integration schemes.  Find the documentation for NumPy's own version of the trapezoidal integration scheme and check its result with your own:

#### Part 6 (extension)

Write a function  `trapzf(f, a, b, npts=100)` that accepts a function `f`, the endpoints `a` and `b` and the number of samples to take `npts`.  Sample the function uniformly at these
points and return the value of the integral.

Use the trapzf function to identify the minimum number of sampling points needed to approximate the integral $\int_0^3 x^2$ with an absolute error of $<=0.0001$. (A loop is necessary here.)